In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:

directory = '/content/gdrive/MyDrive/PatronesComportamiento/Ventanadeslizante'
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm

import os
currentFile=os.path.abspath(os.getcwd())
print(currentFile)
inten=currentFile.split('PatronesComportamiento-Notebooks/')


/home/ana/Documentos/TFG/PatronesComportamiento/Notebooks/PatronesComportamiento-Notebooks/PatronesComportamiento


Para la predicción querremos utilizar el método RandomForest entre otros de la librería SKlearn, para ello será ncesario un formateo de los datos. Convertiremos el problema de predicción en un problema de aprendizaje supervisado. Utilzando un ventana deslizante selecionaremos un número w de muestras que serán las predecesoras y tomaremos la muestra w+1 como consecuencia. La pregunta es: ¿por qué debemos tranformar los datos a este formato? Bien, los datos de los que dispnemos son series temporales con valores contínuos y lo que estamos buscando es una predicción de valores futuros. Los algoritmos que utilizaremos tanto RandomForest como SGBRERegresor funcionan de la siguiente forma: a partir de una x predice una y. Entonces, lo que estamos buscando es convertir nuestro problema de regresión en un problema de clasificación.



Como se muestra en el ejemplo de abajo tendremos que romar un conjunto de muestras de tamaño w, estas serán la x, después el elemento w+1 será la y, desplazamos la ventana una posición, ahora la ventana englobará la y de la ventana anterior, y repetiremos este proceso hasta haber recorrido toda la serie temporal o las series temporales. Para nuestro caso hemos querido trabajar con una serie temporal formada por todas las series temporales cronológicamente para a la hora de la predicción poder introducir vetanas que se encuentran entre los dos días y favorecer la continuidad en el tiempo.


![](https://drive.google.com/uc?export=view&id=18CZ4r1Asxs78pPpcrswIyKj60cRvr4-m)


Vamos a crear un método que dadas n series temporales las divida en m series temporales todas con una longitud w y un valor como consecuente.

In [3]:
with open(str(currentFile)+'/mySeriesInten.pkl', 'rb') as file:
    timeSeries = pickle.load(file)

with open(str(currentFile)+'/singleSerieInten.pkl', 'rb') as file:
    singleSerie = pickle.load(file)

print(len(singleSerie))




196021


In [4]:
def spliting_timeseries(timeSerie,w,h):  #w amplitud de la ventana y h >1 si es multiobjetivo
  X, y = [], []  #inicializamos los arrays que guardarán atributos y variable predicha
  for i in range(len(timeSerie) - w): #bucle para iterar sobre todas las muestras de las series 
    X.append(timeSerie[i : i + w])  #cogemos desde el elemento en el que estamos hasta los w siguientes
    y.append(timeSerie[i + w:i+w+h])  #aquí cogemos desde el w+1 hasta los h siguientes
  return pd.DataFrame(X), pd.Series(y)







Probamos con los valores de w=3 y h=1 como primera prueba.

In [5]:
cond,consec=[],[]
x,y=spliting_timeseries(singleSerie,3,1) #llamo al método
cond.append(x)  #lo añado a una lista de atributos
consec.append(y)  # lo añado a una lista de variables predichas


data_cond=pd.concat(cond)   #lo convierto en un dataframe de pandas
data_consec=pd.concat(consec)   #lo convierto en un dataframe de pandas

Y guardamos los datos con pickle para poder acceder a ellos desde otro notebook en cualquer momento.

In [6]:



with open(str(currentFile)+'/dataCond.pkl', 'wb') as file:
        pickle.dump(data_cond, file)
with open(str(currentFile)+'dataConsec.pkl', 'wb') as file:
        pickle.dump(data_consec, file)

